In [ ]:
# import tensorflow as tf
import os
import numpy as np
import glob
import cv2
import pandas as pd
import itertools
from matplotlib import pyplot as plt
import random 
from tensorflow.contrib.layers import flatten
import tensorflow as tf
from sklearn.utils import shuffle

%matplotlib inline

#### Variables

In [ ]:
img_size = (32,32)
batch_size = 32
epochs = 14
learning_rate = .0001

#### Import data

In [ ]:
green_images = glob.glob("image_data/green/*.png")
yellow_images = glob.glob("image_data/yellow/*.png")
red_images = glob.glob("image_data/red/*.png")
unknown_images = glob.glob("image_data/unknown/*.png")

labels = ["green"] * len(green_images) + ['yellow'] * len(yellow_images) + ['red'] * len(red_images) + ['unknown'] * len(unknown_images)


#### Create data for test, train and validation

In [ ]:
traffic_light_data = pd.DataFrame({"image_path": green_images + yellow_images + red_images + unknown_images,
                                  "labels": labels})

# shuffle df
traffic_light_data = traffic_light_data.sample(frac=1).reset_index(drop=True)

In [ ]:
test_size = int(len(traffic_light_data)  * .25)
validation_size = int(len(traffic_light_data) * .0825)

In [ ]:
test_set = traffic_light_data.loc[:test_size, ]
validation_set = traffic_light_data.loc[test_size:(test_size + validation_size), ]
training_set = traffic_light_data.loc[test_size + validation_size:, ]

print("test set size:", test_set.shape[0])
print("validation set size:", validation_set.shape[0])
print("training set size:", training_set.shape[0])

In [ ]:
# create training set
X_train = []
for i in training_set['image_path']:
    image = plt.imread(i)
    X_train.append(cv2.resize(image, img_size))

X_train = np.array(X_train)
y_train = training_set['labels']

# create validation set
X_valid = []
for i in validation_set['image_path']:
    image = plt.imread(i)
    X_valid.append(cv2.resize(image, img_size))
X_valid = np.array(X_valid)
y_valid = validation_set['labels']
    
# create test set
X_test = []
for i in test_set['image_path']:
    image = plt.imread(i)
    X_test.append(cv2.resize(image, img_size))
X_test = np.array(X_test)
y_test = test_set['labels']    

In [ ]:
def process_img(img):
    h,w,c = img.shape 
    
    # normalize
    img = cv2.normalize(img , img , alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
    # add rotation
    rotattion_angle = random.uniform(-5,5)
    M = cv2.getRotationMatrix2D((h/2,w/2), rotattion_angle, 1)
    img = cv2.warpAffine(img,M,(w,h))

    # flip
    if random.uniform(0,1) > .5:
        img =  cv2.flip(img, 1 )

    # color jitter
    noise_g = np.random.uniform(-.05, .05, (h, w))
    noise_r = np.random.uniform(-.05, .05, (h, w))
    noise_b = np.random.uniform(-.05, .05, (h, w))
    
    zitter = np.zeros_like(img)
    zitter[:,:,0] = noise_r
    zitter[:,:,1] = noise_g
    zitter[:,:,2] = noise_b
    
    noise_added = cv2.add(img, zitter)  
    
    # cap back to [0,1]
    noise_added[noise_added > 1] = 1
    noise_added[noise_added < 0] = 0
       
    return noise_added

#### check what it looks like

In [ ]:
random_image = X_train[random.sample(range(0,len(X_train)),1)]

fig, axs = plt.subplots(1,2)
axs = axs.ravel()
index = random.randint(0, len(X_train))
axs[0].axis('off')
axs[0].set_title('before')
axs[0].imshow(random_image[0])

axs[1].axis('off')
axs[1].set_title('after')
axs[1].imshow(read_and_process_img(random_image[0]))

### Model architecture

In [ ]:
conv2 = 0
conv1 = 0
def LeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    global conv1
    global conv2
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    #dropout
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    #dropout
    fc2 = tf.nn.dropout(fc2, keep_prob)
    
    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
keep_prob = tf.placeholder(tf.float32)

### Train and test model

In [ ]:
rate = 0.001
logits = LeNet(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

# Model Evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

# # Train the model

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training the model...")
    print()
    for i in range(epochs):
        X_train, y_train = shuffle(X_train, y_train)
        # apply deformation to image to augment
        X_train = [process_img(x) for x in X_train]
        for offset in range(0, num_examples, batch_size):
            end = offset + batch_size
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.50})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, 'traffic_signal')
    print("Model saved")

### stuff used for behavioural cloning, generator to make GPU happy

In [ ]:
def get_image(image_list):
    """
    Generator function
    Generator function, saves RAMs by generating data, instead of pushing it all into your memory in one go
    :param image_list:
    :return: images and angles
    """
    ii = 0
    while True:
        images_out = np.ndarray(shape=(batch_size, img_size[0], img_size[1], 3), dtype=float)
        labels_out = np.ndarray(shape=batch_size, dtype=float)

        for j in range(batch_size):
            if ii > batch_size:
                image_list = shuffledf(image_list)
                ii = 0

            labels_out[j] = image_list[ii][1]
            images_out[j] = read_and_process_img(image_list[ii][0])

            ii += 1


        yield images_out, labels_out

In [ ]:
def calc_samples_per_epoch(array_size, batch_size):
    """
    Calculates sample per epoc,
    :param array_size: length of the training set (or training, validation set)
    :param batch_size: Batch size
    :return: Sample size for each epoch
    """
    num_batches = array_size / batch_size
    samples_per_epoch = math.ceil((num_batches / batch_size) * batch_size)
    samples_per_epoch = samples_per_epoch * batch_size
    return samples_per_epoch